# Connect to historic data API

In [18]:
# Free API, no credentials needed
import yfinance as yf

import pandas as pd
import numpy as np

import utils
import requests
from datapackage import Package, Resource

In [8]:
# Constants
TICKERS = {
    "S&P500": "^GSPC"
}

In [9]:
# Load heavy resources here

# Get the constituents of S&P500 
sp500_constituents = utils.load_sp_500_constituents() # Default scrap mode is false

In [13]:
# Arbitrarly for now: select 5 companies
top_5 = sp500_constituents[:5]

In [50]:
# Get the quaterly report release dates of those tickers
results = []
for index, row in top_5.iterrows():
    symbol = row.symbol
    results.append({symbol: yf.Ticker(symbol).get_calendar()})

In [46]:
yf.Ticker("MMM").calendar

,0,1
Earnings Date,2022-10-24 12:30:00,2022-10-28 12:30:00
Earnings Average,2.68,2.68
Earnings Low,2.59,2.59
Earnings High,2.86,2.86
Revenue Average,8814350000,8814350000
Revenue Low,8639000000,8639000000
Revenue High,9069000000,9069000000


In [10]:
# All information here : https://pypi.org/project/yfinance/
user_agent_headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [15]:
def _scrap_previous_earnings(symbol):
    url = f"https://finance.yahoo.com/calendar/earnings?symbol={symbol}"

    data = requests.get(url, headers=user_agent_headers).text

    try:
        data = pd.read_html(data)[0]
        data.replace("-", np.nan, inplace=True)
    except ValueError:
        print(f"[INFO]: No available earnings data for {symbol}")
    return data


test = _scrap_previous_earnings("MMM")


In [17]:
test[0]

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise(%)
0,MMM,3M Co,"Jan 23, 2023, 9 AMEST",-,-,-
1,MMM,3M Co,"Oct 24, 2022, 8 AMEDT",2.68,-,-
2,MMM,3M Company,"Jul 26, 2022, 2 AMEDT",2.42,2.48,+2.65
3,MMM,3M Company,"Apr 26, 2022, 2 AMEDT",2.31,2.65,+14.72
4,MMM,3M Company,"Jan 25, 2022, 1 AMEST",2.01,2.31,+14.87
...,...,...,...,...,...,...
95,MMM,3M Company,"Apr 30, 1999, 12 AMEDT",0.46,0.48,+3.71
96,MMM,3M Company,"Jan 25, 1999, 12 AMEST",0.4,0.43,+7.37
97,MMM,3M Company,"Oct 22, 1998, 12 AMEDT",0.49,0.49,-1.11
98,MMM,3M Company,"Jul 23, 1998, 12 AMEDT",0.46,0.48,+3.6
